In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

## Example 1 and 2: Finding the area of a triange under the tangent of $ \frac {1}{x}$

In [2]:
#define function

def f(x):
    return 1 / x

def f_der(x):
    return (-1) / (x ** 2) 

def tangent_y_intercept(x):
    return f(x) - f_der(x) * x

def tangent_x_intercept(x):
    return ( f_der(x) * x - f(x) ) / f_der(x)

def area_under_line(x):
    return 0.5 * tangent_y_intercept(x) * tangent_x_intercept(x)


In [3]:
def plot_line(m, c, x, ax):
    y = m * x + c
    # ax.plot(x, y)
    return y

In [ ]:
def plot_area_under_tangent_triange(z):

    x = np. linspace(0.1, 2.5, 100)
    y = f(x)
    
    
    fig, ax = plt.subplots()

    ax.plot(x,y)
    line_y = plot_line(f_der(z), tangent_y_intercept(z), x, ax=ax)
    ax.fill_between(x, line_y, where = line_y >= 0)
    ax.set_xlim(0.1, 2.5)
    ax.set_ylim(-1, 7)

    area = area_under_line(z)
    ax.annotate(f"area = {round(area)}", xy = (z, f(z)), arrowprops=dict(arrowstyle='->'), xytext=(1.5, 6))
    
interact(plot_area_under_tangent_triange, z = widgets.FloatSlider(value = 1 , min = 0.1, max = 2.5, step = 0.01))

interactive(children=(FloatSlider(value=1.0, description='z', max=2.5, min=0.1, step=0.01), Output()), _dom_cl…

<function __main__.plot_area_under_tangent_triange(z)>